## API-kald i Python

### Hvordan bruges API'er?

API'er (til databaser) er meget forskellige, men indeholder typisk de samme følgende grundkomponenter:

**Request:** Ligesom al anden kommunikation med internettet, involverer brug af API at sende en *request* (GET eller POST) til en server. 

**Endpoint:** API'er indeholder typisk flere forskellige *endpoints*. Tænk på disse som "haner", man kan koble sig på, for at udtrække data. Endpoints er URL'er, som en request skal sendes til.

**Parameters:** Parametre er de argumenter eller indstillinger, som endpoint accepterer eller forventer. Nogle er krævet for overhovedet at få data tilbage, andre er valgfrie.

**Authentication:** De fleste API'er kræver en eller anden form for autentificering. Man ser typisk to former for autentificering: HTTPS autentificering (brugernavn og adgangskode) og autentificering via "tokens". Tokens er unikke "nøgler" - en samling af tekst, tegn og tal, som for serveren unikt identificerer, hvem der sender henvendelsen.

## Ekesempel 1: Brug af Statistikbankens API

Link til API dokumentation: https://www.dst.dk/en/Statistik/brug-statistikken/muligheder-i-statistikbanken/api

Man kan tilgå data i Statistikbanken via API.

I det følgende ses nogle eksempler på at bruge API'en via Python.

*Bemærk*: Denne API kræver ikke autentificering

### Udtræk data fra Statistikbankens API

In [5]:
import requests

statbank_api = "https://api.statbank.dk/v1/data"  #Endpoint of the data API

param = {'table': 'folk1c',
            'format': 'CSV',
            'variables': [{'code': 'OMRÅDE', 'values': ['101', '851']},  #Request in JSON/dictionary
                                                            {'code': 'ALDER', 'values': ['20-24', '25-29']}]
           }

data_req = requests.post(statbank_api, json=param)  #Sending requests

print(data_req.text)  #Printing the raw text output

﻿OMRÅDE;ALDER;TID;INDHOLD
København;20-24 år;2023K3;64226
København;25-29 år;2023K3;91941
Aalborg;20-24 år;2023K3;22223
Aalborg;25-29 år;2023K3;22098



Data endpointet returnerer komma-separerede værdier som standard (csv).

Dette format er kompatibelt med `pandas` dataframes (`pd.read_csv`):

In [2]:
from io import StringIO
import pandas as pd

dstdata = StringIO(data_req.text)  #Read the data output as raw text
dstdf = pd.read_csv(dstdata, sep=";")  #Read text as csv
dstdf  #Print data

,OMRÅDE,ALDER,TID,INDHOLD
0,København,20-24 år,2023K3,64226
1,København,25-29 år,2023K3,91941
2,Aalborg,20-24 år,2023K3,22223
3,Aalborg,25-29 år,2023K3,22098


In [3]:
dstdf.groupby(['OMRÅDE']).sum()  #Group by municipality and count sum

,ALDER,TID,INDHOLD
OMRÅDE,,,
Aalborg,20-24 år25-29 år,2023K32023K3,44321
København,20-24 år25-29 år,2023K32023K3,156167


## Eksempel 2: Brug af Danmarks Adresser Web API (DAWA)

Link til API dokumentation: https://dawadocs.dataforsyningen.dk/dok/api/

Danmarks Adresser Web API (DAWA) kan bruges til at hente data om samt søge efter adresser i Danmark.

*Bemærk:* Denne API kræver ikke autentificering.

In [29]:
import requests

adress_end = 'https://api.dataforsyningen.dk/adresser' # endpoint til at søge på adresser (API'en har andre endpoints)

# parametre/indstillinger til søgning (se dokumentation)
parameters = {'q': 'Solsort*',  # søg efter adresser der indeholder Solsort
              'kommunekode': '0851'} # søg i kommunekode 0851 (Aalborg)

data_req = requests.get(adress_end, params = parameters)

In [30]:
data_req.status_code # gik henvendelse igennem?

200

In [31]:
data = data_req.json() # data returneres som standard som JSON (tages ud på denne måde)

In [34]:
from pprint import pprint

pprint(data[:1])

[{'adgangsadresse': {'DDKN': {'km1': '1km_6321_556',
                              'km10': '10km_632_55',
                              'm100': '100m_63212_5565'},
                     'adgangspunkt': {'højde': 17.1,
                                      'id': '0a3f509c-b9d5-32b8-e044-0003ba298018',
                                      'kilde': 5,
                                      'koordinater': [9.93105827, 57.03094927],
                                      'nøjagtighed': 'A',
                                      'tekniskstandard': 'TN',
                                      'tekstretning': 131.7,
                                      'ændret': '2007-03-30T23:59:00.000'},
                     'adressebetegnelse': 'Solsortvej 2, 9000 Aalborg',
                     'afstemningsområde': {'href': 'https://api.dataforsyningen.dk/afstemningsomraader/851/36',
                                           'navn': 'Aalborghus Gymnasium',
                                           'nummer':

In [35]:
len(data) # hvor mange hits?

43

In [37]:
data[0].keys() # hvilke oplysninger/variable?

dict_keys(['id', 'kvhx', 'status', 'darstatus', 'href', 'historik', 'etage', 'dør', 'adressebetegnelse', 'adgangsadresse'])

DAWA kan sende data tilbage i forskellige formater med parameter "struktur":

In [41]:
adress_end = 'https://api.dataforsyningen.dk/adresser' # endpoint til at søge på adresser (API'en har andre endpoints)

# parametre/indstillinger til søgning (se dokumentation)
parameters = {'q': 'Solsort*', # søg efter adresser der indeholder Solsort
              'kommunekode': '0851', # søg i kommunekode 0851 (Aalborg)
              'struktur': 'flad'} # send data tilbage i "flad" struktur (data er ikke "nestet" og kan konverteres direkte til tabel)

data_req = requests.get(adress_end, params = parameters)

In [44]:
data = data_req.json() # data tages stadig ud som JSON, nu er det blot et JSON records format

In [45]:
data_df = pd.DataFrame(data)

In [48]:
data_df.head()

,id,status,darstatus,oprettet,ændret,ikrafttrædelse,nedlagt,vejkode,vejnavn,adresseringsvejnavn,...,navngivenvej_id,storkredsnummer,storkredsnavn,valglandsdelsbogstav,valglandsdelsnavn,landsdelsnuts3,landsdelsnavn,kvhx,kvh,betegnelse
0,0a3f50ca-baff-32b8-e044-0003ba298018,1,3,2000-02-05T21:22:49.000,2000-02-05T21:22:49.000,2000-02-05T21:22:49.000,None,7688,Solsortvej,Solsortvej,...,f2d548ba-4501-427d-8b28-089df535780c,10,Nordjylland,C,Midtjylland-Nordjylland,DK050,Nordjylland,08517688___2_______,08517688___2,"Solsortvej 2, 9000 Aalborg"
1,0a3f50ca-bb01-32b8-e044-0003ba298018,1,3,2000-02-05T21:22:49.000,2000-02-05T21:22:49.000,2000-02-05T21:22:49.000,None,7688,Solsortvej,Solsortvej,...,f2d548ba-4501-427d-8b28-089df535780c,10,Nordjylland,C,Midtjylland-Nordjylland,DK050,Nordjylland,08517688___4_______,08517688___4,"Solsortvej 4, 9000 Aalborg"
2,0a3f50ca-bb02-32b8-e044-0003ba298018,1,3,2000-02-05T21:25:36.000,2000-02-05T21:25:36.000,2000-02-05T21:25:36.000,None,7688,Solsortvej,Solsortvej,...,f2d548ba-4501-427d-8b28-089df535780c,10,Nordjylland,C,Midtjylland-Nordjylland,DK050,Nordjylland,08517688___5_______,08517688___5,"Solsortvej 5, 9000 Aalborg"
3,0a3f50ca-bb03-32b8-e044-0003ba298018,1,3,2000-02-05T21:22:49.000,2000-02-05T21:22:49.000,2000-02-05T21:22:49.000,None,7688,Solsortvej,Solsortvej,...,f2d548ba-4501-427d-8b28-089df535780c,10,Nordjylland,C,Midtjylland-Nordjylland,DK050,Nordjylland,08517688___6_______,08517688___6,"Solsortvej 6, 9000 Aalborg"
4,0a3f50ca-bb04-32b8-e044-0003ba298018,1,3,2000-02-05T21:22:49.000,2000-02-05T21:22:49.000,2000-02-05T21:22:49.000,None,7688,Solsortvej,Solsortvej,...,f2d548ba-4501-427d-8b28-089df535780c,10,Nordjylland,C,Midtjylland-Nordjylland,DK050,Nordjylland,08517688___7_______,08517688___7,"Solsortvej 7, 9000 Aalborg"
